<a href="https://colab.research.google.com/github/jinrumiao/titanic-analysis/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C_01%EF%BC%9A%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E5%AD%98%E6%B4%BB%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**1.   載入資料、確認資料內容**


In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


**2.   資料前處理**



In [17]:
from sklearn.preprocessing import StandardScaler

# 移除對於是否存活沒有意義的特徵：PassengerId、Name、Ticket
df.drop(columns=["PassengerId", "Name", "Ticket"], axis=1, inplace=True, errors="ignore")
# 處理類別型資料：Sex、Cabin、Embarked
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

df["Cabin"] = df["Cabin"].notnull().astype(int)

df.dropna(subset=["Embarked"], inplace=True)
one_hot_ec = pd.get_dummies(df["Embarked"], prefix="Embarked")
df = pd.concat([df, one_hot_ec], axis=1)
df.drop("Embarked", axis=1, inplace=True)

# 處理數值型資料：Pclass、Age、SibSp、Parch、Fare
df["Age"].fillna(df["Age"].mean(), inplace=True)

normalize_columns = ["Age", "SibSp", "Parch", "Fare"]
scaler = StandardScaler()
normalized_data = scaler.fit_transform(df[normalize_columns])
normalized_df = pd.DataFrame(normalized_data, columns=normalize_columns)

df[normalize_columns] = normalized_df

df.dropna(how="any", inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 887 entries, 0 to 888
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    887 non-null    int64  
 1   Pclass      887 non-null    int64  
 2   Sex         887 non-null    int64  
 3   Age         887 non-null    float64
 4   SibSp       887 non-null    float64
 5   Parch       887 non-null    float64
 6   Fare        887 non-null    float64
 7   Cabin       887 non-null    int64  
 8   Embarked_C  887 non-null    uint8  
 9   Embarked_Q  887 non-null    uint8  
 10  Embarked_S  887 non-null    uint8  
dtypes: float64(4), int64(4), uint8(3)
memory usage: 65.0 KB


**3.   分離訓練資料與標籤**

In [18]:
df_train = df.copy()

columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']

train_X = df_train[list(columns_X)]
train_y = df_train[columns_y]

train_X.info()
train_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 887 entries, 0 to 888
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Parch       887 non-null    float64
 1   Fare        887 non-null    float64
 2   Embarked_S  887 non-null    uint8  
 3   Age         887 non-null    float64
 4   Embarked_Q  887 non-null    uint8  
 5   Cabin       887 non-null    int64  
 6   Pclass      887 non-null    int64  
 7   SibSp       887 non-null    float64
 8   Sex         887 non-null    int64  
 9   Embarked_C  887 non-null    uint8  
dtypes: float64(4), int64(3), uint8(3)
memory usage: 58.0 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 887 entries, 0 to 888
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Survived  887 non-null    int64
dtypes: int64(1)
memory usage: 13.9 KB


**4.   建立機器學習模型、進行訓練、評估模型**

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(), cv=5, scoring='accuracy')
print(scores)
print(scores.mean())

[0.79213483 0.76966292 0.7740113  0.78531073 0.78531073]
0.7812861042341142
